# Tutorial 11: Reading and writing fields

There are two main file formats to which a `discretisedfield.Field` object can be saved:

- [VTK](https://vtk.org/) for visualisation using e.g., [ParaView](https://www.paraview.org/) or [Mayavi](https://docs.enthought.com/mayavi/mayavi/)
- OOMMF [Vector Field File Format (OVF)](https://math.nist.gov/oommf/doc/userguide12a5/userguide/Vector_Field_File_Format_OV.html) for exchanging fields with micromagnetic simulators.

Let us say we have a nanosphere sample:

$$x^2 + y^2 + z^2 <= r^2$$

with $r=5\,\text{nm}$. The space is discretised into cells with dimensions $(0.5\,\text{nm}, 0.5\,\text{nm}, 0.5\,\text{nm})$. The value of the field at $(x, y, z)$ point is $(-cy, cx, cz)$, with $c=10^{9}$. The norm of the field inside the cylinder is $10^{6}$.

Let us first build that field.

In [1]:
import discretisedfield as df

r = 5e-9
cell = (0.5e-9, 0.5e-9, 0.5e-9)

mesh =  df.Mesh(p1=(-r, -r, -r), p2=(r, r, r), cell=cell)

def norm_fun(pos):
    x, y, z = pos
    if x**2 + y**2 + z**2 <= r**2:
        return 1e6
    else:
        return 0
    
def value_fun(pos):
    x, y, z = pos
    c = 1e9
    return (-c*y, c*x, c*z)

field = df.Field(mesh, value=value_fun, norm=norm_fun)

Let us have a quick view of the field we created

In [3]:
field.plane('z').k3d_vectors(color_field=field.z)

Output()

## Writing the field to a file

The main method used for saving field in different files is `discretisedfield.Field.write()`. It takes `filename` as an argument, which is a string with one of the following extensions:
- `'.vtk'` for saving in the VTK format
- `'.ovf'`, `'.omf'`, `'.ohf'` for saving in the OVF format

Let us firstly save the field in the VTK file.

In [4]:
vtkfilename = 'my_vtk_file.vtk'
field.write(vtkfilename)

We can check if the file was saved in the current directory.

In [8]:
import os
os.path.isfile(f'./{vtkfilename}')

True

Now, we can delete the file:

In [9]:
os.remove(f'./{vtkfilename}')

Next, we can save the field in the OVF format and check whether it was created in the current directory.

In [15]:
omffilename = 'my_omf_file.omf'
field.write(omffilename)

os.path.isfile(f'./{omffilename}')

True

## Reading the OVF file

The method for reading OVF files is a class method `discretisedfield.Field.fromfile()`. By passing a `filename` argument, it reads the file and creates a `discretisedfield.Field` object.

In [16]:
read_field = df.Field.fromfile(omffilename)

Like previouly, we can quickly visualise the field

In [17]:
read_field.plane('z').k3d_vectors(color_field=read_field.z)

Output()

Finally, we can delete the OVF file we created.

In [18]:
os.remove(f'./{omffilename}')

## Other

Full description of all existing functionality can be found in the [API Reference](https://discretisedfield.readthedocs.io/en/latest/api_documentation.html).